In [1]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text, inspect

from postgres_auth import db_writer_url


load_dotenv()


engine = create_engine(db_writer_url)

with engine.connect() as conn:
    result = conn.execute(text("SELECT MAX(obs_time) FROM webpage_plot_data"))
    latest_history = result.scalar()

lower_bound = latest_history if latest_history else '1970-01-01'

query = f"""
    SELECT
        obs.sensor_id,
        sens.location,
        obs.obs_time,
        obs.temp1,
        obs.hum1,
        obs.temp2,
        obs.hum2,
        obs.co2,
        obs.aqi
    FROM observations obs
    JOIN sensors sens
    ON obs.sensor_id = sens.sensor_id
    WHERE obs.obs_time > '{lower_bound}'
"""

df = pd.read_sql_query(query, engine)

In [2]:
df

,sensor_id,location,obs_time,temp1,hum1,temp2,hum2,co2,aqi
0,PICO_W_06,vent,2026-01-12 15:54:38,32.39,21.06,NaN,NaN,NaN,NaN
1,PICO_W_04,bedroom,2026-01-12 15:52:19,28.26,24.20,30.23,22.34,400.0,1.0
2,PICO_W_03,closet,2026-01-12 15:52:21,23.82,29.80,25.09,27.17,NaN,NaN
3,PICO_W_07,thermostat,2026-01-12 15:52:21,27.26,30.82,27.39,29.37,NaN,NaN
4,PICO_W_06,vent,2026-01-12 15:52:21,32.21,21.19,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10835,PICO_W_04,bedroom,2026-01-12 15:53:26,28.24,24.29,30.14,22.26,400.0,1.0
10836,PICO_W_01,office,2026-01-12 15:53:27,29.26,26.76,29.01,25.75,400.0,1.0
10837,PICO_W_06,vent,2026-01-12 15:53:26,32.43,21.05,NaN,NaN,NaN,NaN
10838,PICO_W_03,closet,2026-01-12 15:53:28,23.80,29.85,25.09,27.16,NaN,NaN


In [3]:
df['temp'] = df[['temp1', 'temp2']].mean(axis=1, skipna=True)
df['temp_f'] = df['temp'] * 9 / 5 + 32
df['hum'] = df[['hum1', 'hum2']].mean(axis=1, skipna=True)

df['interval_time'] = pd.to_datetime(df['obs_time']).dt.round('5min')

active_sensors = df['location'].unique()

In [4]:
active_sensors

array(['vent', 'bedroom', 'closet', 'thermostat', 'office'], dtype=object)

In [4]:
df

,sensor_id,location,obs_time,temp1,hum1,temp2,hum2,co2,aqi,temp,temp_f,hum,interval_time
0,PICO_W_07,thermostat,2026-01-12 15:51:11,27.20,30.78,27.35,29.34,NaN,NaN,27.275,81.095,30.060,2026-01-12 15:50:00
1,PICO_W_06,vent,2026-01-12 15:58:05,32.38,21.04,NaN,NaN,NaN,NaN,32.380,90.284,21.040,2026-01-12 16:00:00
2,PICO_W_01,office,2026-01-12 15:50:03,29.21,26.86,29.12,25.86,400.0,1.0,29.165,84.497,26.360,2026-01-12 15:50:00
3,PICO_W_04,bedroom,2026-01-12 15:50:02,28.29,24.35,30.16,22.29,400.0,1.0,29.225,84.605,23.320,2026-01-12 15:50:00
4,PICO_W_06,vent,2026-01-12 15:50:02,32.56,20.93,NaN,NaN,NaN,NaN,32.560,90.608,20.930,2026-01-12 15:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5552,PICO_W_03,closet,2026-01-12 15:52:18,23.81,29.81,25.10,27.18,NaN,NaN,24.455,76.019,28.495,2026-01-12 15:50:00
5553,PICO_W_07,thermostat,2026-01-12 15:52:18,27.28,30.79,27.39,29.36,NaN,NaN,27.335,81.203,30.075,2026-01-12 15:50:00
5554,PICO_W_06,vent,2026-01-12 15:52:18,32.23,21.24,NaN,NaN,NaN,NaN,32.230,90.014,21.240,2026-01-12 15:50:00
5555,PICO_W_01,office,2026-01-12 15:52:20,29.30,26.86,29.09,25.76,400.0,1.0,29.195,84.551,26.310,2026-01-12 15:50:00


In [6]:
agg = (
    df.groupby(['interval_time', 'location'])
        .agg(
            temp=('temp_f', 'mean'),
            hum=('hum', 'mean'),
            co2=('co2', 'mean'),
            aqi=('aqi', 'mean'),
        )
        .reset_index()
        .rename(columns={'interval_time': 'obs_time'})
)

# leave off incomplete data at end
hist_5m = agg[agg['obs_time'] < agg['obs_time'].max()]

In [7]:
wide = hist_5m.pivot(
    index='obs_time',
    columns='location',
    values=['temp', 'hum', 'co2', 'aqi']
).swaplevel(axis=1).sort_index(axis=1).reset_index()

In [8]:
wide

location            obs_time bedroom                                   closet  \
                                 aqi         co2        hum       temp    aqi   
0        2026-01-12 15:50:00     1.0  400.000000  23.384894  84.587574    NaN   
1        2026-01-12 15:55:00     1.0  400.000000  23.384681  84.466840    NaN   
2        2026-01-12 16:00:00     1.0  400.010638  23.486649  84.444915    NaN   
3        2026-01-12 16:05:00     1.0  400.202128  23.485160  84.456213    NaN   
4        2026-01-12 16:10:00     1.0  400.000000  23.371702  84.602319    NaN   
5        2026-01-12 16:15:00     1.0  400.478723  23.418138  84.617351    NaN   
6        2026-01-12 16:20:00     1.0  400.191489  23.515160  84.593128    NaN   
7        2026-01-12 16:25:00     1.0  400.329787  23.480691  84.612468    NaN   
8        2026-01-12 16:30:00     1.0  400.553191  23.485638  84.613521    NaN   
9        2026-01-12 16:35:00     1.0  400.393617  23.441596  84.680255    NaN   
10       2026-01-12 16:40:00     1.0  400.368421  23.465895  84.700589    NaN   
11       2026-01-12 16:45:00     1.0  400.096774  23.354194  84.738839    NaN   

location                              office  ...                        \
         co2        hum       temp       aqi  ...        hum       temp   
0        NaN  28.497979  76.019574  1.000000  ...  26.283511  84.508298   
1        NaN  28.436330  76.010766  1.000000  ...  26.291170  84.464830   
2        NaN  28.463526  76.001758  1.000000  ...  26.410160  84.613234   
3        NaN  28.386011  76.035085  1.010638  ...  27.061596  84.881511   
4        NaN  28.311809  76.048777  1.000000  ...  26.685160  85.036904   
5        NaN  28.362074  76.089947  1.000000  ...  26.988617  85.430989   
6        NaN  28.385426  76.086213  1.000000  ...  26.457181  85.998181   
7        NaN  28.360266  76.145000  1.000000  ...  26.312234  85.464979   
8        NaN  28.356000  76.173611  1.000000  ...  26.418989  85.270809   
9        NaN  28.305798  76.217383  1.000000  ...  26.307766  85.214702   
10       NaN  28.278936  76.271574  1.000000  ...  26.922553  85.507106   
11       NaN  28.336702  76.224277  1.000000  ...  26.635638  85.745128   

location thermostat                           vent                            
                aqi co2        hum       temp  aqi co2        hum       temp  
0               NaN NaN  30.143438  81.086562  NaN NaN  21.083542  90.261125  
1               NaN NaN  29.987312  81.282452  NaN NaN  21.043918  90.333175  
2               NaN NaN  30.017947  81.297168  NaN NaN  21.055670  90.279361  
3               NaN NaN  29.981064  81.331681  NaN NaN  21.235258  89.930124  
4               NaN NaN  29.880426  81.474723  NaN NaN  21.150309  90.111794  
5               NaN NaN  29.828298  81.723755  NaN NaN  20.850208  90.762125  
6               NaN NaN  30.054211  81.393421  NaN NaN  20.775918  91.098041  
7               NaN NaN  29.988710  81.537742  NaN NaN  20.773542  91.064188  
8               NaN NaN  29.911947  81.690137  NaN NaN  20.770000  91.051876  
9               NaN NaN  30.992606  81.836638  NaN NaN  20.828041  90.951113  
10              NaN NaN  30.215851  82.537298  NaN NaN  20.655361  91.301278  
11              NaN NaN  29.975319  82.649223  NaN NaN  20.655729  91.265563  

[12 rows x 21 columns]

In [11]:
wide.columns

Index(['obs_time', 'sensor__bedroom_aqi', 'sensor__bedroom_co2',
       'sensor__bedroom_hum', 'sensor__bedroom_temp', 'sensor__closet_aqi',
       'sensor__closet_co2', 'sensor__closet_hum', 'sensor__closet_temp',
       'sensor__office_aqi', 'sensor__office_co2', 'sensor__office_hum',
       'sensor__office_temp', 'sensor__thermostat_aqi',
       'sensor__thermostat_co2', 'sensor__thermostat_hum',
       'sensor__thermostat_temp', 'sensor__vent_aqi', 'sensor__vent_co2',
       'sensor__vent_hum', 'sensor__vent_temp'],
      dtype='object')

In [10]:

def flatten_col(col_tuple):
    prefix = '' if col_tuple[0] == 'obs_time' else 'sensor__'

    # Drop empty strings and join with underscores
    return prefix + '_'.join([str(x) for x in col_tuple if x])

# flatten cols from multiindex to regular
wide.columns = [flatten_col(col) for col in wide.columns]

In [12]:
conn = engine.begin()

# Check for and add any missing columns
inspector = inspect(engine)
existing_cols = {col['name'] for col in inspector.get_columns('webpage_plot_data')}
df_cols = set(wide.columns) - {'obs_time'}
missing_cols = df_cols - existing_cols

In [13]:
missing_cols

set()

In [15]:
min_interval = wide['obs_time'].min()

min_interval


Timestamp('2026-01-12 14:15:00')

In [16]:
pd.read_sql_query("""
    SELECT * FROM webpage_plot_data
    WHERE obs_time >= '2026-01-12 14:15:00'
""", engine)

,obs_time,sensor__bedroom_hum,sensor__bedroom_temp,sensor__closet_hum,sensor__closet_temp,sensor__kitchen_hum,sensor__kitchen_temp,sensor__office_hum,sensor__office_temp,sensor__vent_temp,...,sensor__vent_co2,sensor__closet_aqi,sensor__couch_co2,sensor__thermostat_hum,sensor__bedroom_aqi,sensor__thermostat_temp,sensor__vent_hum,sensor__kitchen_aqi,sensor__couch_temp,sensor__vent_aqi
0,2026-01-12 14:15:00,23.704786,83.832344,29.184946,75.02584,None,None,28.207872,83.323265,90.377711,...,None,None,None,34.16,1.0,77.2412,20.982371,None,None,None


In [19]:
with engine.begin() as conn:
    conn.execute(
        text("""
            DELETE FROM webpage_plot_data
            WHERE obs_time >= :min_interval
        """),
        {"min_interval": min_interval}
    )
    wide.to_sql('webpage_plot_data', conn, if_exists='append', index=False)

In [6]:
pico4 = df[df['sensor_id']=='PICO_W_04']
pico4['obs_time']

2      2026-01-12 14:15:02
6      2026-01-12 14:15:05
10     2026-01-12 14:15:08
14     2026-01-12 14:15:11
18     2026-01-12 14:17:44
               ...        
9418   2026-01-12 14:18:39
9423   2026-01-12 14:18:42
9428   2026-01-12 14:18:45
9432   2026-01-12 14:18:48
9436   2026-01-12 14:18:51
Name: obs_time, Length: 1881, dtype: datetime64[ns]

In [7]:
# Verify actual monotonicity
is_monotonic = pico4.sort_values('obs_time').equals(pico4.reset_index(drop=True))
print(f"Data is sorted: {is_monotonic}")

# Or directly check
print(pico4['obs_time'].is_monotonic_increasing)

Data is sorted: False
False
